In [32]:
import pandas as pd
import numpy as numpy
import numpy as np

In [33]:
tipos_conv = {'USER_ID':np.int32,
              'convirtio':np.bool,
              "cantidad_paginas_vistas_trimestre": np.int64,
             "cant_paginas_vistas_en_mes_1": np.int64,
             "cant_paginas_vistas_en_mes_2": np.int64,
              "cant_paginas_vistas_en_mes_3": np.int64}

In [34]:
train=pd.read_csv("../data/train_completo.csv")#,dtype=tipos_conv)

In [35]:
train.head()

,USER_ID,convirtio,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3
0,6574,False,1141.0,524.0,364.0,253.0
1,2639,True,671.0,261.0,179.0,231.0
2,3898,False,219.0,88.0,50.0,81.0
3,7996,False,383.0,95.0,178.0,110.0
4,6111,False,425.0,208.0,93.0,124.0


In [42]:
Y=train['convirtio']
Y.head()

0    False
1     True
2    False
3    False
4    False
Name: convirtio, dtype: bool

In [41]:
X=train[['cantidad_paginas_vistas_trimestre','cant_paginas_vistas_en_mes_1','cant_paginas_vistas_en_mes_2','cant_paginas_vistas_en_mes_3']]
X.head()

,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3
0,1141.0,524.0,364.0,253.0
1,671.0,261.0,179.0,231.0
2,219.0,88.0,50.0,81.0
3,383.0,95.0,178.0,110.0
4,425.0,208.0,93.0,124.0


In [40]:
X=X.fillna(0)
X=X.astype(int)
X.head()

,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3
0,1141,524,364,253
1,671,261,179,231
2,219,88,50,81
3,383,95,178,110
4,425,208,93,124


In [ ]:
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(objective="binary:logistic", random_state=42)

In [ ]:
xgb_model.fit(X, Y)

In [ ]:

#y_pred = xgb_model.predict(X)
#print(confusion_matrix(Y, y_pred))

In [11]:
xgb_model.score(X,Y)

0.8347537383246806

In [44]:
predecir=pd.read_csv("../data/sampleSubmission.csv")#,dtype=tipos_conv)

In [45]:
predecir_pageviews=pd.read_csv("../data/features/predecir_pageviews.csv")

In [46]:
predecir_pageviews.head()

,USER_ID,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3
0,6574,1290,278.0,299.0,713.0
1,2639,836,288.0,204.0,344.0
2,3898,530,213.0,181.0,136.0
3,7996,150,70.0,47.0,33.0
4,6111,323,121.0,78.0,124.0


In [47]:
predecir_final = predecir.merge(predecir_pageviews,on = "USER_ID", how = 'left')

In [48]:
predecir_final.head()

,USER_ID,SCORE,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3
0,0,0,1519,527.0,490.0,502.0
1,1,0,477,138.0,140.0,199.0
2,2,0,651,222.0,142.0,287.0
3,3,0,381,63.0,121.0,197.0
4,4,0,627,284.0,278.0,65.0


In [49]:
predecir_final=predecir_final[['cantidad_paginas_vistas_trimestre','cant_paginas_vistas_en_mes_1','cant_paginas_vistas_en_mes_2','cant_paginas_vistas_en_mes_3']]
predecir_final=predecir_final.fillna(0)
predecir_final=predecir_final.astype(int)
predecir_final.head()

,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3
0,1519,527,490,502
1,477,138,140,199
2,651,222,142,287
3,381,63,121,197
4,627,284,278,65


In [18]:
y_predict=xgb_model.predict(predecir_final)

In [20]:
predecir['SCORE']=y_predict

In [27]:
predecir.set_index('USER_ID',inplace=True)
predecir.head()

,SCORE
USER_ID,
0,0.053
1,0.006
2,0.065
3,0.006
4,0.007


In [28]:
predecir.to_csv('../data/submit_xgb.csv')